In [ ]:
import os
import json # 今回は直接使用しませんが、構造化データ連携の際に有用
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI # Google Geminiモデル用

# .envファイルから環境変数を読み込む
load_dotenv()

print("ライブラリのインポートと環境変数のロードが完了しました。")

In [ ]:
# APIキーの読み込み
api_key = os.getenv("GEMINI_API_KEY")

if not api_key:
    print("エラー: 環境変数 'GEMINI_API_KEY' が .env ファイルに設定されていません。")
    print(".env ファイルを作成し、GEMINI_API_KEY=\"YOUR_KEY_HERE\" のように記述してください。")
    # このセル以降の実行を中断させるために例外を発生させるか、条件分岐で制御
    raise ValueError("Gemini APIキーが設定されていません。処理を中断します。")
else:
    print("Gemini APIキーが正常に読み込まれました。")

# LLMクライアントの初期化
# 注意: 'gemini-2.0-flash' というモデル名は2025年5月時点で一般的でない可能性があります。
# 利用可能なモデル名（例: 'gemini-1.5-flash-latest', 'gemini-pro'など）を適宜確認・変更してください。
# エラーが出る場合は、モデル名をGoogle Cloudのドキュメントで確認してください。
try:
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash", # ユーザー指定は "gemini-2.0-flash" でしたが、より一般的なモデル名に変更。必要に応じて調整してください。
        google_api_key=api_key,
        temperature=0, # ユーザー指定のデフォルトtemperature
        # convert_system_message_to_human=True # 必要に応じてシステムメッセージの扱いを調整
    )
    print(f"ChatGoogleGenerativeAIクライアントがモデル '{llm.model}' で初期化されました。")
except Exception as e:
    print(f"LLMクライアントの初期化中にエラーが発生しました: {e}")
    print("モデル名が正しいか、APIキーに十分な権限があるか確認してください。")
    llm = None # エラー時はllmをNoneに設定

In [ ]:
def call_llm(prompt_text: str, temperature: float = None, max_tokens: int = None) -> str:
    """
    初期化されたLLMクライアントを使用してプロンプトを実行し、応答を取得する関数。
    temperature や max_tokens は呼び出し時に指定があれば、LLM初期化時の設定を上書き試行します。
    """
    if llm is None:
        return "Error: LLMクライアントが初期化されていません。"

    try:
        # LangchainのChatGoogleGenerativeAIはinvokeメソッドの引数で直接temperatureやmax_tokensを
        # 簡単には上書きできない場合があります。
        # 通常は初期化時に設定するか、GenerationConfigを使います。
        # ここでは、初期化時の設定を基本としつつ、もし個別に設定したい場合は
        # 新しいLLMインスタンスをその都度生成するか、より高度な設定方法を検討します。
        # 今回はシンプルに、初期化時の設定で動作させます。
        # もし個別の呼び出しでtemperatureやmax_output_tokensを変更したい場合は、以下のようにします:
        
        current_model_kwargs = llm.model_kwargs if llm.model_kwargs else {}
        generation_config = {}
        if temperature is not None:
            generation_config['temperature'] = temperature
        if max_tokens is not None:
            generation_config['max_output_tokens'] = max_tokens
        
        if generation_config:
            # invokeにgeneration_configを渡せるか確認 (LangChainのバージョンやモデルによる)
            # もし直接渡せない場合は、llm.with_structured_outputやllm.bindなどで設定変更が必要な場合あり
            # ここでは簡略化のため、初期化時の設定が優先されることを前提とします。
            # ChatGoogleGenerativeAI の invoke に直接 generation_config を渡すのは標準的ではない。
            # .bind() や .with_options() を使うのが一般的
            
            # invoke時に設定を適用する例 (ただし、これは ChatGoogleGenerativeAI の標準的な使い方とは異なる可能性あり)
            # response = llm.invoke(prompt_text, generation_config=generation_config)
            
            # より確実なのは、新しい設定でインスタンスを一時的に作るか、bindを使う
            # 例: llm.bind(generation_config=generation_config).invoke(prompt_text)
            # ここでは簡単のため、初期化設定を使い、引数は無視する形で進めます。
            # 必要であれば、より高度な設定方法を実装してください。
            if temperature is not None and llm.temperature != temperature:
                 print(f"  call_llm: temperature引数({temperature})は現在無視され、初期化時の値({llm.temperature})が使用されます。")
            # max_tokensについては、ChatGoogleGenerativeAIの初期化では直接指定せず、
            # generate_config内のmax_output_tokensで制御するため、ここでは特に触れません。

        response = llm.invoke(prompt_text)
        return response.content.strip()

    except Exception as e:
        print(f"LLM呼び出し中にエラーが発生しました: {e}")
        return f"Error: LLMからの応答取得に失敗しました。詳細: {e}"

# テスト呼び出し (任意)
# test_prompt = "こんにちは、今日の調子はどうですか？"
# test_response = call_llm(test_prompt)
# print(f"テストプロンプト: {test_prompt}")
# print(f"テスト応答: {test_response}")

In [ ]:
class ThoughtNode:
    """思考の木における各ノードを表すクラス"""
    def __init__(self, text: str, parent=None, evaluation: str = None, id_path: str =""):
        self.text = text.strip()
        self.parent = parent
        self.children: list[ThoughtNode] = []
        self.evaluation = evaluation
        self.id_path = id_path

    def add_child(self, child_node):
        self.children.append(child_node)

    def __repr__(self, level=0) -> str:
        eval_text = f" (評価: {self.evaluation})" if self.evaluation else ""
        id_text = f" (ID: {self.id_path})" if self.id_path else ""
        ret = "\t" * level + repr(self.text) + eval_text + id_text + "\n"
        for child in self.children:
            ret += child.__repr__(level + 1)
        return ret

print("ThoughtNode クラスが定義されました。")

In [ ]:
def generate_initial_thoughts(problem: str, num_thoughts: int = 3) -> list[str]:
    """与えられたお題に対して、初期の複数の思考（アイデア）を生成する"""
    prompt = f"""
    お題: 「{problem}」

    このお題について、{num_thoughts}個の異なる初期アイデアや思考の方向性をリスト形式で提案してください。
    各アイデアは簡潔に記述し、それぞれ独立した行に記述してください。

    例:
    - アイデアA
    - アイデアB
    - アイデアC

    あなたの提案:
    """
    # LLM初期化時のtemperatureが0なので、ここではtemperatureを指定しない
    response = call_llm(prompt_text=prompt, max_tokens=num_thoughts * 50)
    if response.startswith("Error:") : return [response]
    thoughts = [line.strip().lstrip('- ').lstrip('* ') for line in response.split('\n') if line.strip()]
    return thoughts[:num_thoughts] if thoughts else ["生成されたアイデアがありませんでした。"]

def expand_thought(thought_text: str, problem: str, current_path_str: str, num_children: int = 2) -> list[str]:
    """特定の思考をさらに深掘りし、サブアイデアや次のステップを生成する"""
    prompt = f"""
    現在のお題: 「{problem}」
    現在の思考の文脈 (パス: {current_path_str}): 「{thought_text}」

    上記の思考「{thought_text}」をさらに具体的に展開し、{num_children}個の異なる詳細なサブアイデア、関連する問い、または次の具体的なアクションステップをリスト形式で提案してください。
    各提案は簡潔に記述し、それぞれ独立した行に記述してください。

    あなたの提案:
    """
    response = call_llm(prompt_text=prompt, max_tokens=num_children * 70)
    if response.startswith("Error:") : return [response]
    sub_thoughts = [line.strip().lstrip('- ').lstrip('* ') for line in response.split('\n') if line.strip()]
    return sub_thoughts[:num_children] if sub_thoughts else ["生成されたサブアイデアがありませんでした。"]

def evaluate_thought_viability(thought_text: str, problem: str, current_path_str: str) -> str:
    """思考の有望性や妥当性をLLMに評価させる（簡易版）"""
    prompt = f"""
    お題: 「{problem}」
    現在の思考パス: {current_path_str}
    検討中の思考: 「{thought_text}」

    この思考「{thought_text}」は、お題「{problem}」の解決や探求において、どの程度有望ですか？
    評価結果を「有望」「中立」「低リスク」「高リスク」「要改善」「興味深い」などの短いキーワードで述べてください。
    可能であれば、その簡単な理由も付け加えてください。

    評価: [キーワード] (理由: [簡単な理由])
    """
    # 評価なので、もしtemperatureを調整したければここで指定（ただし上記call_llmの実装による）
    response = call_llm(prompt_text=prompt, temperature=0.2, max_tokens=100) # 評価なので低めのtemperatureを試みる
    if response.startswith("Error:"): return "評価エラー"
    return response

print("ヘルパー関数 (generate_initial_thoughts, expand_thought, evaluate_thought_viability) が定義されました。")

In [ ]:
def tree_of_thoughts_generator(
    initial_problem: str,
    max_depth: int = 2,
    initial_thoughts_count: int = 3,
    children_per_node: int = 2,
    use_evaluation: bool = False
) -> ThoughtNode:
    """Tree of Thoughtsを生成するメイン関数"""
    if llm is None:
        print("LLMクライアントが初期化されていないため、Tree of Thoughtsを生成できません。")
        return ThoughtNode("Error: LLM not initialized.")

    root_node = ThoughtNode(initial_problem, id_path="0")
    root_node.evaluation = "初期お題"

    queue: list[tuple[ThoughtNode, int]] = [(root_node, 0)]
    node_id_counters: dict[str, int] = {"0": 0}
    visited_texts: set[str] = {initial_problem.lower()}
    processed_nodes_count = 0

    while queue:
        current_node, depth = queue.pop(0)
        processed_nodes_count += 1
        print(f"\nProcessing node (depth {depth}, ID: {current_node.id_path}): {current_node.text[:80]}...")

        if depth >= max_depth:
            print("  Max depth reached for this branch.")
            continue

        generated_ideas: list[str] = []
        if depth == 0:
            print(f"  Generating initial {initial_thoughts_count} thoughts for the problem...")
            generated_ideas = generate_initial_thoughts(current_node.text, initial_thoughts_count)
        else:
            print(f"  Expanding thought into {children_per_node} sub-thoughts...")
            generated_ideas = expand_thought(
                current_node.text,
                initial_problem,
                current_node.id_path,
                children_per_node
            )
        
        print(f"    Generated {len(generated_ideas)} ideas: {generated_ideas}")

        parent_id_path = current_node.id_path
        if parent_id_path not in node_id_counters:
            node_id_counters[parent_id_path] = 0

        for idea_text in generated_ideas:
            if not idea_text or idea_text.startswith("Error:") or idea_text.lower() in visited_texts:
                print(f"    Skipping duplicate or invalid idea: {idea_text[:80]}")
                continue
            visited_texts.add(idea_text.lower())

            child_id_num = node_id_counters[parent_id_path]
            node_id_counters[parent_id_path] += 1
            new_id_path = f"{parent_id_path}-{child_id_num}"

            evaluation_result = None
            if use_evaluation and depth < max_depth :
                print(f"    Evaluating sub-thought (ID: {new_id_path}): {idea_text[:80]}...")
                evaluation_result = evaluate_thought_viability(idea_text, initial_problem, new_id_path)
                print(f"      Evaluation: {evaluation_result}")

            child_node = ThoughtNode(idea_text, parent=current_node, evaluation=evaluation_result, id_path=new_id_path)
            current_node.add_child(child_node)
            
            if depth + 1 < max_depth:
                 queue.append((child_node, depth + 1))
            elif depth + 1 == max_depth:
                 print(f"  Reached max depth for child: {child_node.text[:80]}")

    print(f"\nTree generation complete. Processed {processed_nodes_count} nodes.")
    return root_node

print("tree_of_thoughts_generator 関数が定義されました。")

In [ ]:
def format_to_markdown_mindmap(node: ThoughtNode, level: int = 0, include_eval: bool = True) -> str:
    """思考の木をMarkdownのマインドマップ形式（Markmap互換）に変換する"""
    markdown = ""
    indent = "  " * level 
    prefix = "# " if level == 0 else "- "
    text_to_display = node.text.replace("\n", " ") # 複数行は1行に
    if include_eval and node.evaluation:
        eval_short = node.evaluation.split('\n')[0] # 評価が複数行なら最初の行のみ
        text_to_display += f"  <small>(評価: {eval_short})</small>" # Markmapで小さく表示

    markdown += f"{indent}{prefix}{text_to_display}\n"
    for child in node.children:
        markdown += format_to_markdown_mindmap(child, level + 1, include_eval)
    return markdown

print("format_to_markdown_mindmap 関数が定義されました。")

In [ ]:
def get_final_answer_from_tree(root_node: ThoughtNode, initial_problem: str, max_paths_to_consider: int = 5) -> str:
    """生成された思考の木から有望な経路を抽出し、LLMに最終的な結論を生成させる"""
    if llm is None:
        return "Error: LLMクライアントが初期化されていないため、最終結論を生成できません。"
        
    paths: list[list[ThoughtNode]] = []
    def find_paths_to_leaves(node: ThoughtNode, current_path: list[ThoughtNode]):
        current_path.append(node)
        if not node.children: # 葉ノード
            paths.append(list(current_path)) # パスのコピーを保存
        else:
            for child in node.children:
                find_paths_to_leaves(child, current_path)
        current_path.pop() # バックトラック

    find_paths_to_leaves(root_node, [])
    if not paths: return "思考の木から有効な思考経路が見つかりませんでした。"

    path_texts_for_llm = []
    # 経路を評価や深さでソートするなど、より高度な選択も可能
    for path in paths[:max_paths_to_consider]:
        # ルートノード（お題自身）はパス説明に含めないことが多い
        path_str = " -> ".join([node.text for node in path[1:]]) # path[0]はルート
        if path_str: # 空のパスは含めない
             path_texts_for_llm.append(path_str)

    if not path_texts_for_llm: return "最終結論を生成するための有効な思考経路が抽出できませんでした。"

    context_for_final_answer = "以下は、お題「" + initial_problem + "」についてTree of Thoughtsの手法で展開された主要な思考の経路です。\n\n"
    for i, path_text in enumerate(path_texts_for_llm):
        context_for_final_answer += f"経路 {i+1}: {path_text}\n"
    
    context_for_final_answer += "\nこれらの思考の経路全体を踏まえて、お題「" + initial_problem + "」に対する最も包括的で洞察に富んだ最終的な結論、または複数の選択肢がある場合はそれらを整理した提案をまとめてください。"

    prompt = f"""
    お題: {initial_problem}

    提供された思考の木（Tree of Thoughts）から抽出された主要な思考経路:
    {context_for_final_answer}

    上記の分析結果に基づき、お題に対する最終的な結論や提案を、構造的かつ分かりやすくまとめてください。
    複数のアイデアがある場合は、それらを統合したり、比較したり、優先順位を示したりすることも検討してください。
    結論は、具体的なアクションや次のステップに繋がるような形で提示することが望ましいです。
    """
    print("\n--- LLMに最終結論の生成を依頼中 ---")
    # 結論生成時はtemperatureを少し上げて多様性を出すことも検討できる
    final_answer = call_llm(prompt_text=prompt, temperature=0.5, max_tokens=1500)
    return final_answer

print("get_final_answer_from_tree 関数が定義されました。")

In [ ]:
if __name__ == "__main__": # Jupyter Notebookではこのブロックは通常直接実行されないが、
                           # スクリプトとして実行された場合のために残しておいても良い
    print("メイン実行ロジックを開始します (Jupyter Notebookではこのセルを直接実行してください)。")

# --- Jupyter Notebookで直接実行する部分 ---
if llm is None:
    print("LLMクライアントが初期化されていないため、処理を実行できません。セル2を再確認してください。")
else:
    # ▼▼▼ お題を直接ここに記述してください ▼▼▼
    initial_problem_prompt = "機械学習の基礎"
    # 例: "再生可能エネルギーの普及を加速するための斬新な政策提言"
    # 例: "2030年までに日本の食品ロスを半減させるための実行可能な戦略"
    # ▲▲▲ お題の記述はここまで ▲▲▲

    print(f"設定されたお題: 「{initial_problem_prompt}」")

    if not initial_problem_prompt.strip():
        print("エラー: お題が設定されていません。上記の initial_problem_prompt にお題を記述してください。")
        # 必要に応じて処理を中断
        # exit() # Jupyter Notebookではexit()はカーネルを停止させる可能性があるので注意
    else:
        # ToTのパラメータ設定
        max_depth_setting = 2        # 木の深さ (0はお題自身。1で初期思考、2でサブ思考)
        initial_thoughts_count = 3   # 初期に生成する思考の数
        children_per_node_count = 2  # 各思考から展開するサブ思考の数
        enable_evaluation = False    # 各思考の評価を行うか (TrueにするとLLMコール数が増加)

        print(f"\nお題「{initial_problem_prompt}」についてTree of Thoughtsを開始します...")
        print(f"設定: 深さ={max_depth_setting}, 初期思考数={initial_thoughts_count}, 各ノードの子思考数={children_per_node_count}, 評価={enable_evaluation}\n")

        # Tree of Thoughtsを生成
        thought_tree_root = tree_of_thoughts_generator(
            initial_problem_prompt,
            max_depth=max_depth_setting,
            initial_thoughts_count=initial_thoughts_count,
            children_per_node=children_per_node_count,
            use_evaluation=enable_evaluation
        )

        if thought_tree_root.text.startswith("Error:"):
            print(f"Tree of Thoughtsの生成に失敗しました: {thought_tree_root.text}")
        else:
            print("\n\n--- 生成された思考の木 (内部構造プレビュー) ---")
            for i, child in enumerate(thought_tree_root.children):
                print(f"  初期思考 {i+1}: {child.text}")
                for j, grandchild in enumerate(child.children):
                    print(f"    サブ思考 {i+1}-{j+1}: {grandchild.text}")

            # Markdownマインドマップ形式に変換
            markdown_output = format_to_markdown_mindmap(thought_tree_root, include_eval=enable_evaluation)

            print("\n\n--- マインドマップ用Markdown出力 ---")
            print(markdown_output) # Notebookのセルに出力

            # Markdownファイルを保存
            filename_md = "thought_tree_mindmap.md"
            try:
                with open(filename_md, "w", encoding="utf-8") as f:
                    f.write(markdown_output)
                print(f"\nマインドマップが '{filename_md}' に保存されました。")
                print("このファイルをMarkmap (例: https://markmap.js.org/repl ) やVSCodeのMarkdown Preview Enhanced拡張機能などで開くと、視覚的なマインドマップとして表示できます。")
            except Exception as e:
                print(f"Markdownファイルの保存中にエラーが発生しました: {e}")

            # 生成された思考の木に基づいて最終的な結論をLLMに生成させる
            print("\n\n--- Tree of Thoughtsに基づく最終結論の生成 ---")
            final_conclusion = get_final_answer_from_tree(thought_tree_root, initial_problem_prompt)
            
            print("\n\n--- 最終結論 ---")
            print(final_conclusion) # Notebookのセルに出力

            # 最終結論をファイルに保存
            filename_conclusion = "final_conclusion.txt"
            try:
                with open(filename_conclusion, "w", encoding="utf-8") as f:
                    f.write(f"お題: {initial_problem_prompt}\n\n")
                    f.write("生成された思考の木 (Markdownマインドマップ形式):\n")
                    f.write(markdown_output)
                    f.write("\n\n---\n\n")
                    f.write("最終結論:\n")
                    f.write(final_conclusion)
                print(f"\n最終結論と思考プロセスが '{filename_conclusion}' に保存されました。")
            except Exception as e:
                print(f"最終結論ファイルの保存中にエラーが発生しました: {e}")